In [ ]:
!echo -e "beautifulsoup4==4.8.2\ncertifi==2020.4.5.1\nfeedparser==5.2.1\niso8601==0.1.12\nscrapy==2.0.0\nstanza==1.0.1\ntldextract==2.2.2\ntorch==1.3.1\n" > requirements.txt

In [ ]:
!pip install -r requirements.txt

In [ ]:
import stanza
stanza.download('en')

In [ ]:
import logging
import stanza
from bs4 import BeautifulSoup
logging.basicConfig(level=logging.DEBUG)
# This class gives the match rule for traversal for template given parsed beautiful soup object and author_name
# and given a traversal rule identify the author
class TraversalRule:
    NER_TAG_PERSON = "PERSON"
    nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

    def __init__(self, parsed_page, author_name, traversal_rule):
        self.parsed_page = parsed_page
        self.author_name = author_name
        self.traversal_rule = list()
        if traversal_rule:
            self.traversal_rule = traversal_rule

    # https://stackoverflow.com/questions/54265391/find-all-end-nodes-that-contain-text-using-beautifulsoup4
    def mark_if_leaf_with_text(self, node):
        if node.name in ["style", "script", "link", "meta"]:
            return False
        if not node.text:
            return False
        elif len(node.find_all(text=False)) > 0:  # no other tags inside other than text
            return False
        node.leaf = True
        return False

    def is_leaf_nodes_with_people(self, node):
        if not node.leaf:
            return False
        text = node.text
        processed_text = self.nlp(text)
        no_of_entities = len(processed_text.entities)
        people = [ent for ent in processed_text.entities if ent.type == self.NER_TAG_PERSON]
        no_of_people = len(people)
        self.candidates = list()
        if no_of_people > 0:
            print(no_of_people, processed_text.entities)
            node.no_of_people = no_of_people
            node.no_of_entities = no_of_entities
            node.people = people
            return True
        return False

    def find_leaf_nodes_with_people(self, node):
        self.mark_if_leaf_with_text(node)
        return self.is_leaf_nodes_with_people(node)

    def find_candidates(self):
        logging.info("Finding Candidate Authors")
        # Use stanza to identify people in text, the nodes containing them and traversal for the leaf nodes
        candidate_author_nodes_details = self.parsed_page.find_all(self.find_leaf_nodes_with_people)
        candidate_authors = list()
        for node in candidate_author_nodes_details:
            candidate_author = dict()
            # This condition may not hold
            if node.no_of_people == 1:
                candidate_author['author_entity'] = node.people[0].text
                candidate_author['ancestors'] = [parent.name for parent in node.parents]
                candidate_author['node_name'] = node.name
                candidate_authors.append(candidate_author)
        self.candidates = candidate_authors

    def pick_traversal_from_author(self):
        logging.info("Picking the traversal rule given author")
        self.find_candidates()
        for candidate in self.candidates:
            candidate_author_name = candidate['author_entity']
            normal_name = self.get_normal_name()
            logging.info("Candidate Author Name: {}, normal name {}".format(candidate_author_name, normal_name))
            if candidate_author_name == self.author_name or candidate_author_name == normal_name:
                self.traversal_rule = candidate['ancestors']

    def get_author_from_traversal(self):
        logging.info("Picking the author from candidates based on the traversal rule")

        self.find_candidates()
        candidates = self.candidates
        if candidates:
            for candidate in candidates:
                if candidate['ancestors'] == self.traversal_rule:
                    return candidate['author_entity']

        return None


    def get_normal_name(self):
        if "," in self.author_name:
            split = self.author_name.split(",")
            not_last_name = split[1]
            last_name = split[0]
            normal_name = not_last_name + " " + last_name
            return normal_name.strip()
        return "None"

In [ ]:
with open("./resources/linkedin-origin-ab-testing-nicolai-kramer-jakobsen.html", "r", encoding="UTF-8") as fp:
    html_content = fp.read()

soup = BeautifulSoup(html_content, 'lxml')
soup = BeautifulSoup(soup.prettify('utf-8'), 'lxml')  # some inputs are so messy that they affect the output
t_rule = TraversalRule(soup, "Nicolai Kramer Jakobsen", None)
t_rule.pick_traversal_from_author()
print(t_rule.traversal_rule)

In [1]:
import urllib3
from bs4 import BeautifulSoup
import json
import tldextract
import certifi
import ssl

ssl_context = ssl.SSLContext()
ssl_context.load_verify_locations(certifi.where())
http = urllib3.PoolManager(ssl_context=ssl_context)

class AuthorTraversalRules:
    persistence_type = "json"

    def __init__(self, filename):
        self.filename = filename
        self.author_traversal_rules = dict()
        self.load_author_traversal_rules()

    def load_author_traversal_rules(self):
        with open(self.filename, "r") as fp:
            self.author_traversal_rules =  json.load(fp)

    def get_author_traversal_for_url(self, url):
        extract_result = tldextract.extract(url)
        host_url = extract_result.registered_domain
        if host_url in self.author_traversal_rules:
            return self.author_traversal_rules[host_url]
        return None

class FindAuthorWithTraversal:

    def __init__(self, url, author_traversal_rule_for_site):
        self.url = url
        self.author_traversal_rule = author_traversal_rule_for_site
        self.page_content = None

    def load_page_content(self):
        self.page_content = http.request('GET', self.url).data

    def get_author(self):
        self.load_page_content()
        soup = BeautifulSoup(self.page_content, 'lxml')
        soup = BeautifulSoup(soup.prettify('utf-8'), 'lxml')
        t = TraversalRule(soup, None, self.author_traversal_rule)
        return t.get_author_from_traversal()


class FindAuthor:
    domain_traversal_file = "./resources/domain_traversal_rules-500.json"
    domain_traversal = AuthorTraversalRules(domain_traversal_file)

    def __init__(self, url):
        self.url = url
        extracted = tldextract.extract(url)
        site = extracted.registered_domain
        print(site)
        self.find_author = FindAuthorWithTraversal(self.url, self.domain_traversal.author_traversal_rules[site])

    def get_author(self):
        return self.find_author.get_author()

In [2]:
print(FindAuthor("https://www.linkedin.com/pulse/automating-user-creation-aws-sftp-service-transfer-arjun-dandagi/").get_author())

linkedin.com


c:\users\meena\appdata\local\programs\python\python36\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.linkedin.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


NameError: name 'TraversalRule' is not defined